In [1]:
import numpy as np
import pandas as pd
import zipcodes

# DMAs to ZIPs

In [3]:
# load generic ZIP / DMA file
zip_to_dma = pd.read_csv('data/zip_to_dma', sep='\t')
zip_to_dma.head(5)

,zip_code,dma_code,dma_description
0,1001,543,SPRINGFIELD - HOLYOKE
1,1002,543,SPRINGFIELD - HOLYOKE
2,1003,543,SPRINGFIELD - HOLYOKE
3,1004,543,SPRINGFIELD - HOLYOKE
4,1005,506,BOSTON (MANCHESTER)


In [5]:
# load test set DMA file
dma = pd.read_csv('data/dma_test_set.csv')
dma.rename(columns={'DMA': 'dma_code', 'DMA Name': 'dma_description'}, inplace=True)
dma.head(5)

,dma_code,dma_description
0,528,MIAMI - FT. LAUDERDALE
1,635,AUSTIN
2,602,CHICAGO
3,753,PHOENIX (PRESCOTT)
4,504,PHILADELPHIA


In [7]:
# gut check - check that 528 is Miami
zip_to_dma[zip_to_dma['dma_code'] == 528].head(5)

,zip_code,dma_code,dma_description
13620,33001,528,MIAMI - FT. LAUDERDALE
13621,33002,528,MIAMI - FT. LAUDERDALE
13622,33004,528,MIAMI - FT. LAUDERDALE
13623,33008,528,MIAMI - FT. LAUDERDALE
13624,33009,528,MIAMI - FT. LAUDERDALE


In [8]:
dma_with_zips = pd.merge(left=dma, right=zip_to_dma, on='dma_code', how='inner')
dma_with_zips = dma_with_zips[['dma_code', 'dma_description_x', 'zip_code']]
dma_with_zips.rename(columns={'dma_description_x': 'dma_name'}, inplace=True)

In [9]:
# unique zip codes in our file -- matches the test set!
dma_with_zips['dma_code'].unique()

array([528, 635, 602, 753, 504, 770, 534, 862, 505, 512, 535, 561, 804,
       567, 544, 533, 790, 622, 563, 757, 811, 630, 686, 679, 556, 669,
       749, 866, 566, 529, 754, 523, 550, 507, 678, 658, 765, 541, 637,
       801, 514, 633, 821, 648, 716, 693, 577, 513, 800, 868, 592, 675,
       547, 656, 642, 760, 702, 743, 509, 530, 574, 600, 698, 588, 531,
       543, 687, 610, 564, 582, 705, 598, 537, 526, 603, 569, 734, 767,
       759, 661, 755, 558, 581, 606, 631, 643, 639, 553, 554, 711, 647,
       597, 740])

In [10]:
dma_with_zips.head(5)

,dma_code,dma_name,zip_code
0,528,MIAMI - FT. LAUDERDALE,33001
1,528,MIAMI - FT. LAUDERDALE,33002
2,528,MIAMI - FT. LAUDERDALE,33004
3,528,MIAMI - FT. LAUDERDALE,33008
4,528,MIAMI - FT. LAUDERDALE,33009


# ZIPs to City & State

In [11]:
example_zip = '94110'
city = zipcodes.matching(example_zip)[0]['city']
state = zipcodes.matching(example_zip)[0]['state']
print(city, state)

San Francisco CA


In [13]:
# convert zip code to either city or state
def zip_to_location_info(zip_code: str, info_key: str) -> str:
    # check that we're requesting valid info
    if info_key not in ['city', 'state']:
        print("Invalid information to request")
     
    # left pad string with 0s if less than 5 digits
    if len(zip_code) < 5:
        zip_code = zip_code.zfill(5)
    
    # check if zip code is real
    if zipcodes.is_real(zip_code):
        zip_dict = zipcodes.matching(zip_code)[0]
        return zip_dict[info_key]
    else:
        return None

In [14]:
example_zip = '94110'
print(zip_to_location_info(example_zip, 'city'))
print(zip_to_location_info(example_zip, 'state'))

San Francisco
CA


In [15]:
example_zip = '111'
print(zip_to_location_info(example_zip, 'city'))
print(zip_to_location_info(example_zip, 'state'))

None
None


In [ ]:
dma_with_zips['city'] = dma_with_zips['zip_code'].apply(lambda x: zip_to_location_info(str(x), 'city'))
dma_with_zips['state'] = dma_with_zips['zip_code'].apply(lambda x: zip_to_location_info(str(x), 'state'))

In [ ]:
# 16694 rows
dma_with_zips.head(5)

In [ ]:
# drop 10 rows that aren't valid ZIPs - 16684 rows
dma_with_city_state = dma_with_zips[~dma_with_zips['city'].isna()]

In [ ]:
dma_with_city_state.to_csv('data/dma_with_city_state.csv', index=False)

# Debug Nulls

Not really needed, but just some debugging to confirm that the set of zip codes that returned no information are in fact not real zip codes.

In [91]:
invalid_zips = dma_with_zips[dma_with_zips['city'].isna()]

In [93]:
invalid_zips

,dma_code,dma_name,zip_code,city,state
655,602,CHICAGO,60296,None,None
656,602,CHICAGO,60297,None,None
738,602,CHICAGO,60483,None,None
1567,504,PHILADELPHIA,8677,None,None
7024,679,DES MOINES - AMES,50350,None,None
14368,610,ROCKFORD,61058,None,None
14667,564,CHARLESTON - HUNTINGTON,25697,None,None
16022,581,TERRE HAUTE,47813,None,None
16023,581,TERRE HAUTE,47814,None,None
16436,554,WHEELING - STEUBENVILLE,26186,None,None


In [82]:
invalid_zips.to_csv('data/invalid_zips.csv')

In [74]:
example_zip = '60296'

In [78]:
zipcodes.is_real(example_zip)

False

In [79]:
zipcodes.similar_to(example_zip)

[]